### Set Up for Project Imports


In [1]:
import sys
from pathlib import Path

In [2]:
# Add repo root (one level up from 'source/')
ROOT = Path.cwd().parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [3]:
# Auto-reload code changes
%load_ext autoreload
%autoreload 2

## Imports

In [24]:
from data.api import UcIrvineAPI
from data.dataset_id import DatasetIDs
import json
import pandas

## Data

In [11]:
apartment_for_rent_classified = UcIrvineAPI.fetch_dataset(repo_id= DatasetIDs.Apartment_For_Rent_Classified.value)

/Users/Isaia/PycharmProjects/CS-577-07-Data-Science-Fall-2025-Project/.venv/lib/python3.12/site-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (0,5,6,12,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [18]:
print(json.dumps(apartment_for_rent_classified.metadata, indent=2))

{
  "uci_id": 555,
  "name": "Apartment for Rent Classified",
  "repository_url": "https://archive.ics.uci.edu/dataset/555/apartment+for+rent+classified",
  "data_url": "https://archive.ics.uci.edu/static/public/555/data.csv",
  "abstract": "This is a dataset of classified for apartments for rent in USA.\n",
  "area": "Business",
  "tasks": [
    "Classification",
    "Regression",
    "Clustering"
  ],
  "characteristics": [
    "Multivariate"
  ],
  "num_instances": 10000,
  "num_features": 21,
  "feature_types": [
    "Categorical",
    "Integer"
  ],
  "demographics": [],
  "target_col": null,
  "index_col": [
    "id"
  ],
  "has_missing_values": "no",
  "missing_values_symbol": null,
  "year_of_dataset_creation": 2019,
  "last_updated": "Mon Feb 26 2024",
  "dataset_doi": "10.24432/C5X623",
  "creators": [],
  "intro_paper": null,
  "additional_info": {
    "summary": "The dataset contains of 10'000 or 100'000 rows and of 22 columns The data has been cleaned in the way that \r\nc